In [1]:
import sys
import os

project_root = os.getcwd()  
if project_root not in sys.path:
    sys.path.append(project_root)
    
from classes.experiment_config import ExperimentConfig
from classes.experiment_runner import ExperimentRunner

import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)

from accelerate import notebook_launcher
from datasets import load_dataset


INFO:datasets:PyTorch version 2.1.2 available.


In [2]:
experiment_config = ExperimentConfig(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    is_encoder_decoder="decoder_only",
    task_type="text_generation",
    inference_type="pure_generative",  # must match your ExperimentRunner expectations
    max_input_tokens=512,
    max_output_tokens=50,
    num_input_prompts=5,
    gpu_list=[0, 1],
    num_processes=2,
    batching_options={
        "adaptive_batching": True,
        "max_batch_size": 8,
        "adaptive_max_tokens": 256  # adjust as needed
    },
    sharding_config={
        "fsdp_config": {
            "use_orig_params": True,
            "cpu_offload": True
        },
        "sharding_strategy": "NO_SHARD"
    },
    query_rate=1,
    decoder_temperature=1,
    fp_precision="float16",
    backend="pytorch"
)

# Load prompts (here, using the 'arxiv' split from the lighteval/pile_helm dataset)
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
prompts = [sample["text"] for sample in ds]

# Define a function to run the experiment.
def run_experiment():
    runner = ExperimentRunner(experiment_config, prompts)
    runner.run_torch()

# Launch the experiment across the specified number of processes.
notebook_launcher(run_experiment, num_processes=experiment_config.num_processes)

Launching training on 2 GPUs.


INFO:classes.experiment_runner:TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded using backend pytorch with precision float16.
INFO:classes.experiment_runner:TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded using backend pytorch with precision float16.
INFO:classes.experiment_runner:[Process 140170] Model is on device: cuda:1
INFO:classes.experiment_runner:[Process 140169] Model is on device: cuda:0


Original generate method reassigned


INFO:classes.experiment_runner:[Process 140169] Dummy forward pass complete.
INFO:classes.experiment_runner:[Process 140170] Dummy forward pass complete.


Allocated GPUs: GPU 0: NVIDIA A100-PCIE-40GB, GPU 1: NVIDIA A100-PCIE-40GB


...

Prompts processed: 5 prompts. Longest prompt has 71508 characters.


[codecarbon WARNING @ 20:53:44] Multiple instances of codecarbon are allowed to run at the same time.


Energy tracking started
pure_generative task type


[2025-03-18 20:53:49,984] torch.distributed.elastic.multiprocessing.api: [ERROR] failed (exitcode: 1) local_rank: 0 (pid: 140169) of fn: run_experiment (start_method: fork)


ChildFailedError: 
============================================================
run_experiment FAILED
------------------------------------------------------------
Failures:
  <NO_OTHER_FAILURES>
------------------------------------------------------------
Root Cause (first observed failure):
[0]:
  time      : 2025-03-18_20:53:49
  host      : ds01
  rank      : 0 (local_rank: 0)
  exitcode  : 1 (pid: 140169)
  error_file: /tmp/torchelastic_xj7llea8/none_89a5ee9j/attempt_0/0/error.json
  traceback : Traceback (most recent call last):
    File "/home/228755@hertie-school.lan/thesis/thesis/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 346, in wrapper
      return f(*args, **kwargs)
    File "/tmp/ipykernel_140013/939049554.py", line 36, in run_experiment
      runner.run_torch()
    File "/home/228755@hertie-school.lan/thesis/classes/experiment_runner.py", line 109, in run_torch
      self.raw_text_outputs, input_ids, raw_inference_resul